🚨 Set the correct RAW_FILE path

- The cell below will attempt to check your Downloads folder for any file with a name that contains "Crimes_-_2001_to_Present".

    - If you know the file path already, you can skip the next cell and just manually set the RAW_FILE variable in the following code cell.

In [1]:
## Run the cell below to attempt to programmatically find your crime file
import os,glob

## Getting the home folder from environment variables
home_folder = os.environ['HOME']
# print("- Your Home Folder is: " + home_folder)

## Check for downloads folder
if 'Downloads' in os.listdir(home_folder):
    
    
    # Print the Downloads folder path
    dl_folder = os.path.abspath(os.path.join(home_folder,'Downloads'))
    print(f"- Your Downloads folder is '{dl_folder}/'\n")
    
    ## checking for crime files using glob
    crime_files = sorted(glob.glob(dl_folder+'/**/Crimes_-_2001_to_Present*',recursive=True))
    
    # If more than 
    if len(crime_files)==1:
        RAW_FILE = crime_files[0]
        
    elif len(crime_files)>1:
        print('[i] The following files were found:')
        
        for i, fname in enumerate(crime_files):
            print(f"\tcrime_files[{i}] = '{fname}'")
        print(f'\n- Please fill in the RAW_FILE variable in the code cell below with the correct filepath.')

else:
    print(f'[!] Could not programmatically find your downloads folder.')
    print('- Try using Finder (on Mac) or File Explorer (Windows) to navigate to your Downloads folder.')

- Your Downloads folder is '/Users/heng-tsertsai/Downloads/'



In [2]:
## (Required) MAKE SURE TO CHANGE THIS VARIABLE TO MATCH YOUR LOCAL FILE NAME
RAW_FILE = "/Users/heng-tsertsai/Downloads/Crimes_-_2001_to_Present.csv" #(or slice correct index from the crime_files list)

if RAW_FILE != "/Users/heng-tsertsai/Downloads/Crimes_-_2001_to_Present.csv":
	raise Exception("You must update the RAW_FILE variable to match your local filepath.")
	
RAW_FILE

'/Users/heng-tsertsai/Downloads/Crimes_-_2001_to_Present.csv'

In [3]:
## (Optional) SET THE FOLDER FOR FINAL FILES
OUTPUT_FOLDER = 'Data/Chicago/'
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

**Full Workflow**
- Now that your RAW_FILE variable is set either:
    - On the toolbar, click on the Kernel menu > "Restart and Run All".
    - OR click on this cell first, then on the toolbar click on the "Cell" menu > "Run All Below"

In [4]:
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format',lambda x: f"{x:,.2f}")

In [5]:
chicago_full = pd.read_csv(RAW_FILE)
chicago_full

/var/folders/dj/tq7kx_ys6m161zqrzvh_73q80000gn/T/ipykernel_13646/346346930.py:1: DtypeWarning: Columns (0,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  chicago_full = pd.read_csv(RAW_FILE)


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,631.00,6.00,8.00,44.00,06,NaN,NaN,"2,018.00",04/06/2019 04:04:43 PM,NaN,NaN,NaN
1,11645836,JC212333,05/01/2016 12:25:00 AM,055XX S ROCKWELL ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,False,False,824.00,8.00,15.00,63.00,11,NaN,NaN,"2,016.00",04/06/2019 04:04:43 PM,NaN,NaN,NaN
2,11449702,JB373031,07/31/2018 01:30:00 PM,009XX E HYDE PARK BLVD,2024,NARCOTICS,POSS: HEROIN(WHITE),STREET,True,False,233.00,2.00,5.00,41.00,18,NaN,NaN,"2,018.00",04/09/2019 04:24:58 PM,NaN,NaN,NaN
3,11643334,JC209972,12/19/2018 04:30:00 PM,056XX W WELLINGTON AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,"2,514.00",25.00,31.00,19.00,14,NaN,NaN,"2,018.00",04/04/2019 04:16:11 PM,NaN,NaN,NaN
4,11645527,JC212744,02/02/2015 10:00:00 AM,069XX W ARCHER AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,OTHER,False,False,811.00,8.00,23.00,56.00,11,NaN,NaN,"2,015.00",04/06/2019 04:04:43 PM,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7029999,8665323,HV322970,06/07/2012 01:30:00 PM,058XX S NARRAGANSETT AVE,1812,NARCOTICS,POSS: CANNABIS MO{,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7030000,"""error"" : true",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7030001,"""message"" : ""Internal error""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7030002,"""status"" : 500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# explicitly setting the format to speed up pd.to_datetime
date_format = "%m/%d/%Y %I:%M:%S %p"


### Demonstrating/testing date_format
example = chicago_full.loc[0,'Date']
display(example)
pd.to_datetime(example,format=date_format)

'09/01/2018 12:01:00 AM'

Timestamp('2018-09-01 00:01:00')

In [7]:
# this cell can take up to 1 min to run
chicago_full['Datetime'] = pd.to_datetime(chicago_full['Date'], format=date_format)
chicago_full = chicago_full.sort_values('Datetime')
chicago_full = chicago_full.set_index('Datetime')
chicago_full

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
Datetime,,,,,,,,,,,,,,,,,,,,,,
2001-01-01 00:00:00,3723059,HK830095,01/01/2001 12:00:00 AM,071XX S UNION AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,APARTMENT,False,False,732.00,7.00,6.00,68.00,06,"1,172,939.00","1,857,735.00","2,001.00",03/31/2006 10:03:38 PM,41.77,-87.64,"(41.7650536769, -87.6416861711)"
2001-01-01 00:00:00,1322043,G003560,01/01/2001 12:00:00 AM,061XX S ARTESIAN AV,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,825.00,8.00,NaN,NaN,14,"1,161,130.00","1,863,925.00","2,001.00",08/17/2015 03:03:40 PM,41.78,-87.68,"(41.7822923254, -87.684798685)"
2001-01-01 00:00:00,1968773,HH161620,01/01/2001 12:00:00 AM,057XX N SHERIDAN RD,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,"2,022.00",20.00,NaN,NaN,06,"1,168,626.00","1,938,660.00","2,001.00",08/17/2015 03:03:40 PM,41.99,-87.66,"(41.9872120869, -87.6551508725)"
2001-01-01 00:00:00,5952122,HN751199,01/01/2001 12:00:00 AM,027XX W MEDILL AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,APARTMENT,False,False,"1,431.00",14.00,1.00,22.00,06,"1,157,520.00","1,915,449.00","2,001.00",12/21/2007 01:04:15 AM,41.92,-87.70,"(41.9237537709, -87.6966333598)"
2001-01-01 00:00:00,1312179,G000960,01/01/2001 12:00:00 AM,048XX W VAN BUREN ST,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,"1,533.00",15.00,NaN,NaN,14,"1,144,373.00","1,897,538.00","2,001.00",08/17/2015 03:03:40 PM,41.87,-87.75,"(41.874861694, -87.7453919443)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-26 23:59:00,13120824,JG317572,06/26/2023 11:59:00 PM,070XX N CLARK ST,031A,ROBBERY,ARMED - HANDGUN,RESTAURANT,False,False,"2,423.00",24.00,49.00,1.00,03,"1,163,431.00","1,946,667.00","2,023.00",07/03/2023 05:11:09 PM,42.01,-87.67,"(42.0092948032, -87.6740314243)"
NaT,"""error"" : true",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,"""message"" : ""Internal error""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
(chicago_full.isna().sum()/len(chicago_full)).round(2)

ID                     0.00
Case Number            0.00
Date                   0.00
Block                  0.00
IUCR                   0.00
Primary Type           0.00
Description            0.00
Location Description   0.00
Arrest                 0.00
Domestic               0.00
Beat                   0.00
District               0.00
Ward                   0.09
Community Area         0.09
FBI Code               0.00
X Coordinate           0.01
Y Coordinate           0.01
Year                   0.00
Updated On             0.00
Latitude               0.01
Longitude              0.01
Location               0.01
dtype: float64

# Separate the Full Dataset by Years

In [9]:
# save the years for every crime
chicago_full["Year"] = chicago_full.index.year
chicago_full["Year"] = chicago_full["Year"].astype(str)
chicago_full["Year"] = chicago_full["Year"].str.split('.').str[0]
chicago_full["Year"].value_counts()

2002    486799
2001    485754
2003    475972
2004    469402
2005    453753
2006    448168
2007    437068
2008    427161
2009    392816
2010    369798
2011    350698
2016    269802
2017    269087
2018    268796
2019    261272
2022    238643
2015    218582
2020    212153
2021    208681
2012    153025
2023    120491
2014      6752
2013      5327
nan          4
Name: Year, dtype: int64

In [10]:
## Dropping unneeded columns to reduce file size
drop_cols = ["X Coordinate","Y Coordinate", "Community Area","FBI Code",
             "Case Number","Updated On",'Block','Location','IUCR']

In [11]:
# save final df
chicago_final = chicago_full.drop(columns=drop_cols).sort_index()#.reset_index()
chicago_final

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Year,Latitude,Longitude
Datetime,,,,,,,,,,,,,
2001-01-01 00:00:00,3723059,01/01/2001 12:00:00 AM,THEFT,FINANCIAL ID THEFT: OVER $300,APARTMENT,False,False,732.00,7.00,6.00,2001,41.77,-87.64
2001-01-01 00:00:00,1311351,01/01/2001 12:00:00 AM,BURGLARY,UNLAWFUL ENTRY,FACTORY/MANUFACTURING BUILDING,False,False,925.00,9.00,NaN,2001,NaN,NaN
2001-01-01 00:00:00,3182190,01/01/2001 12:00:00 AM,THEFT,FINANCIAL ID THEFT: OVER $300,APARTMENT,False,False,"2,213.00",22.00,21.00,2001,41.71,-87.66
2001-01-01 00:00:00,1318802,01/01/2001 12:00:00 AM,THEFT,$500 AND UNDER,RESIDENCE,False,True,532.00,5.00,NaN,2001,41.67,-87.62
2001-01-01 00:00:00,1370270,01/01/2001 12:00:00 AM,THEFT,$500 AND UNDER,PARKING LOT/GARAGE(NON.RESID.),False,False,"1,512.00",15.00,NaN,2001,41.89,-87.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-26 23:59:00,13120824,06/26/2023 11:59:00 PM,ROBBERY,ARMED - HANDGUN,RESTAURANT,False,False,"2,423.00",24.00,49.00,2023,42.01,-87.67
NaT,"""error"" : true",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN
NaT,"""message"" : ""Internal error""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN


In [12]:
chicago_final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7030004 entries, 2001-01-01 00:00:00 to NaT
Data columns (total 13 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    object 
 1   Date                  object 
 2   Primary Type          object 
 3   Description           object 
 4   Location Description  object 
 5   Arrest                object 
 6   Domestic              object 
 7   Beat                  float64
 8   District              float64
 9   Ward                  float64
 10  Year                  object 
 11  Latitude              float64
 12  Longitude             float64
dtypes: float64(5), object(8)
memory usage: 750.9+ MB


In [13]:
chicago_final.memory_usage(deep=True).astype(float)

Index                   56,240,032.00
ID                     253,574,423.00
Date                   555,370,128.00
Primary Type           471,417,564.00
Description            514,367,855.00
Location Description   479,039,469.00
Arrest                 232,223,220.00
Domestic               228,829,640.00
Beat                    56,240,032.00
District                56,240,032.00
Ward                    56,240,032.00
Year                   428,830,240.00
Latitude                56,240,032.00
Longitude               56,240,032.00
dtype: float64

In [14]:
# unique # of year bins
year_bins = chicago_final['Year'].astype(str).unique()
year_bins

array(['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022', '2023', 'nan'],
      dtype=object)

In [15]:
FINAL_DROP = ['Datetime','Year']#,'Location Description']

In [16]:
## set save location 

os.makedirs(OUTPUT_FOLDER, exist_ok=True)
print(f"[i] Saving .csv's to {OUTPUT_FOLDER}")
## loop through years
for year in year_bins:
    
    ## save temp slices of dfs to save.
    temp_df = chicago_final.loc[ year]
    temp_df = temp_df.reset_index(drop=False)
    temp_df = temp_df.drop(columns=FINAL_DROP)

    # save as csv to output folder
    fname_temp = f"{OUTPUT_FOLDER}Chicago-Crime_{year}.csv"#.gz
    temp_df.to_csv(fname_temp,index=False)

    print(f"- Succesfully saved {fname_temp}")

[i] Saving .csv's to Data/Chicago/
- Succesfully saved Data/Chicago/Chicago-Crime_2001.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2002.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2003.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2004.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2005.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2006.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2007.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2008.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2009.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2010.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2011.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2012.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2013.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2014.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2015.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2016.csv
- Succesfully saved Data/Chicago/Chicago-Crime_2017.csv
- Succesfully

KeyError: ''

In [ ]:
saved_files = sorted(glob.glob(OUTPUT_FOLDER+'*.*csv'))
saved_files

In [ ]:
## create a README.txt for the zip files
readme = """Source URL: 
- https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2
- Filtered for years 2000-Present.

Downloaded 07/18/2022
- Files are split into 1 year per file.

EXAMPLE USAGE:
>> import glob
>> import pandas as pd
>> folder = "Data/Chicago/"
>> crime_files = sorted(glob.glob(folder+"*.csv"))
>> df = pd.concat([pd.read_csv(f) for f in crime_files])
"""
print(readme)


with open(f"{OUTPUT_FOLDER}README.txt",'w') as f:
    f.write(readme)

# Confirmation

In [ ]:
# get list of files from folder
crime_files = sorted(glob.glob(OUTPUT_FOLDER+"*.csv"))
df = pd.concat([pd.read_csv(f) for f in crime_files])
df

In [ ]:
years = df['Date'].map(lambda x: x.split()[0].split('/')[-1])
years.value_counts().sort_index()